In [1]:
pip install yara-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import yara

# Define a YARA rule (you can write your own or load from a file)
rules = yara.compile(source='''
rule example_rule {
    strings:
        $a = "malicious_string"
    condition:
        $a
}
''')

# Scan a file
matches = rules.match('E:\letter to bank.docx')

# Check for matches
if matches:
    for match in matches:
        print(f"Match found: {match}")
else:
    print("No matches found")


No matches found


In [4]:
import hashlib

def calculate_file_hashes(file_path):
    """
    Calculate the SHA-256 and MD5 hashes of a file.

    Args:
        file_path (str): Path to the file.

    Returns:
        tuple: SHA-256 hash and MD5 hash as strings.
    """
    sha256_hash = hashlib.sha256()
    md5_hash = hashlib.md5()
    try:
        with open(file_path, "rb") as file:
            while chunk := file.read(8192):
                sha256_hash.update(chunk)
                md5_hash.update(chunk)
        return sha256_hash.hexdigest(), md5_hash.hexdigest()
    except FileNotFoundError:
        print("Error: File not found.")
        return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def main():
    file_path = input("Enter the path of the file to calculate hashes: ").strip()
    sha256, md5 = calculate_file_hashes(file_path)
    if sha256 and md5:
        print(f"SHA-256: {sha256}")
        print(f"MD5: {md5}")
    else:
        print("Failed to calculate hashes.")

if __name__ == "__main__":
    main()


Enter the path of the file to calculate hashes: E:\letter to bank.docx
SHA-256: 09a2c902dc0224a8cf0e900452d941c48fe6b76348ee21698dee553deb73b77c
MD5: b73e27c0107a69cfbfe7e0bd27c32a82


In [6]:
import hashlib

def generate_sha256(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid memory issues with large files
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

# Example usage:
file_path = 'E:\letter to bank.docx'
print(f"SHA-256 hash of the file: {generate_sha256(file_path)}")


SHA-256 hash of the file: 09a2c902dc0224a8cf0e900452d941c48fe6b76348ee21698dee553deb73b77c


In [9]:
import hashlib
import yara
import os

def calculate_file_hash(file_path):
    """Calculate the SHA-256 hash of a file."""
    sha256_hash = hashlib.sha256()
    try:
        with open(file_path, "rb") as file:
            while chunk := file.read(8192):  # Read in chunks of 8 KB
                sha256_hash.update(chunk)
        return sha256_hash.hexdigest()
    except Exception as e:
        print(f"Error calculating hash for {file_path}: {e}")
        return None

# Example YARA rule (doesn't rely on sha256(file))
yara_rule = '''
rule MaliciousFile {
    meta:
        description = "Detect file based on the presence of malware_signature"
        author = "Your Name"

    strings:
        $malicious_string = "malware_signature" nocase
        $unicode_string = "malware_signature" wide
    condition:
        $malicious_string or $unicode_string
}
'''

# Compile the YARA rule
rules = yara.compile(source=yara_rule)

def scan_file(file_path, expected_hash):
    """Scan a file using YARA and check its SHA-256 hash."""
    # Step 1: Compute SHA-256 hash
    file_hash = calculate_file_hash(file_path)
    if file_hash is None:
        return

    # Step 2: Check if the hash matches the expected value
    if file_hash == expected_hash:
        print(f"File {file_path} matches the expected SHA-256 hash!")
    else:
        print(f"File {file_path} does NOT match the expected SHA-256 hash!")

    # Step 3: Perform YARA scanning
    try:
        matches = rules.match(file_path)
        if matches:
            print(f"YARA rules matched for {file_path}:")
            for match in matches:
                print(f"- Rule: {match.rule}")
        else:
            print(f"No YARA matches for {file_path}")
    except Exception as e:
        print(f"Error scanning {file_path}: {e}")

# Example usage
test_file = "E:\letter to bank.docx"  # Replace with your file path
expected_sha256 = "9bb60008532c360df39c0247f424161bebd856b6da2a8b932793cca65c9f8293"  # Replace with expected hash

scan_file(test_file, expected_sha256)


File E:\letter to bank.docx does NOT match the expected SHA-256 hash!
No YARA matches for E:\letter to bank.docx


In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox
import hashlib
import yara

# Function to calculate SHA-256 and MD5 hashes
def calculate_file_hashes(file_path):
    try:
        sha256_hash = hashlib.sha256()
        md5_hash = hashlib.md5()
        with open(file_path, "rb") as file:
            while chunk := file.read(8192):
                sha256_hash.update(chunk)
                md5_hash.update(chunk)
        return sha256_hash.hexdigest(), md5_hash.hexdigest()
    except Exception as e:
        messagebox.showerror("Error", f"Error calculating hash: {e}")
        return None, None

# Function to validate hashes
def validate_hashes():
    input_hash = input_hash_entry.get().strip()
    file_path = file_path_entry.get().strip()

    if not file_path:
        messagebox.showerror("Error", "Please select a file.")
        return
    if not input_hash:
        messagebox.showerror("Error", "Please enter a hash value to validate.")
        return

    sha256, md5 = calculate_file_hashes(file_path)
    if not sha256 or not md5:
        return

    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, f"SHA-256: {sha256}\nMD5: {md5}\n\n")

    if input_hash.lower() == sha256.lower() or input_hash.lower() == md5.lower():
        result_text.insert(tk.END, "Result: Match Found!\n")
        messagebox.showinfo("Validation Result", "The file matches the input hash!")
    else:
        result_text.insert(tk.END, "Result: No Match Found!\n")
        messagebox.showwarning("Validation Result", "The file does not match the input hash.")

# Function to browse and select a file
def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        file_path_entry.delete(0, tk.END)
        file_path_entry.insert(0, file_path)

# Create the main GUI window
root = tk.Tk()
root.title("YARA Hash Validator")
root.geometry("600x400")

# File selection
file_path_label = tk.Label(root, text="Enter File Name:")
file_path_label.pack(pady=5)
file_path_entry = tk.Entry(root, width=60)
file_path_entry.pack(pady=5)
browse_button = tk.Button(root, text="Browse", command=browse_file)
browse_button.pack(pady=5)

# Input hash
input_hash_label = tk.Label(root, text="Enter Hash to Validate:")
input_hash_label.pack(pady=5)
input_hash_entry = tk.Entry(root, width=60)
input_hash_entry.pack(pady=5)

# Validate button
validate_button = tk.Button(root, text="Validate Hash", command=validate_hashes)
validate_button.pack(pady=10)

# Result display
result_label = tk.Label(root, text="Result:")
result_label.pack(pady=5)
result_text = tk.Text(root, height=10, width=70)
result_text.pack(pady=5)

# Run the application
root.mainloop()


In [3]:
import hashlib

def calculate_file_hashes(file_path):
    sha256_hash = hashlib.sha256()
    md5_hash = hashlib.md5()
    try:
        with open(file_path, "rb") as file:
            while chunk := file.read(8192):
                sha256_hash.update(chunk)
                md5_hash.update(chunk)
        return sha256_hash.hexdigest(), md5_hash.hexdigest()
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def main():
    file_path = input("Enter the file path: ")
    input_hash = input("Enter the hash to validate: ").strip()

    sha256, md5 = calculate_file_hashes(file_path)
    if sha256 and md5:
        print(f"SHA-256: {sha256}")
        print(f"MD5: {md5}")

        if input_hash.lower() in [sha256.lower(), md5.lower()]:
            print("Result: Match Found!")
        else:
            print("Result: No Match Found!")
    else:
        print("Error: Could not calculate hashes.")

if __name__ == "__main__":
    main()


Enter the file path: E:\letter to bank.docx
Enter the hash to validate: 9bb60008532c360df39c0247f424161bebd856b6da2a8b932793cca65c9f8293
SHA-256: 09a2c902dc0224a8cf0e900452d941c48fe6b76348ee21698dee553deb73b77c
MD5: b73e27c0107a69cfbfe7e0bd27c32a82
Result: No Match Found!


In [ ]:
import hashlib
import yara

# Predefined YARA rules
YARA_RULES = {
    "Malicious String Rule": '''
    rule MaliciousString {
        meta:
            description = "Detects files containing the string 'malware_signature'"
        strings:
            $malicious_string = "malware_signature"
        condition:
            $malicious_string
    }
    ''',
    "Suspicious File Rule": '''
    rule SuspiciousFile {
        meta:
            description = "Detects files containing a suspicious hex pattern"
        strings:
            $suspicious_hex = { 4D 5A 90 00 03 00 00 00 }  // Example hex pattern
        condition:
            $suspicious_hex
    }
    ''',
    "PE File Rule": '''
    rule PEFile {
        meta:
            description = "Detects PE files based on their header"
        condition:
            uint16(0) == 0x5A4D  // Checks for "MZ" magic number in PE files
    }
    '''
}

# Function to calculate file hashes (SHA-256 and MD5)
def calculate_file_hashes(file_path):
    sha256_hash = hashlib.sha256()
    md5_hash = hashlib.md5()
    try:
        with open(file_path, "rb") as file:
            while chunk := file.read(8192):
                sha256_hash.update(chunk)
                md5_hash.update(chunk)
        return sha256_hash.hexdigest(), md5_hash.hexdigest()
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Function to validate file against a selected YARA rule
def yara_scan(file_path, yara_rule_content):
    try:
        # Compile the YARA rule
        rules = yara.compile(source=yara_rule_content)
        # Match the file against the YARA rule
        matches = rules.match(file_path)
        return matches
    except yara.SyntaxError as e:
        print(f"YARA Syntax Error: {e}")
        return None
    except Exception as e:
        print(f"Error scanning file with YARA: {e}")
        return None

# Main CLI program
def main():
    print("=== File Hash and YARA Scanner ===")
    file_path = input("Enter the file path: ").strip()
    input_hash = input("Enter the hash to validate (SHA-256 or MD5): ").strip()

    # Calculate file hashes
    sha256, md5 = calculate_file_hashes(file_path)
    if not sha256 or not md5:
        print("Error: Could not calculate hashes. Exiting.")
        return

    # Display file hashes
    print(f"\nFile Hashes:\nSHA-256: {sha256}\nMD5: {md5}\n")

    # Validate hash
    if input_hash.lower() in [sha256.lower(), md5.lower()]:
        print("Hash Validation Result: Match Found!")
    else:
        print("Hash Validation Result: No Match Found!")

    # Display predefined YARA rules
    print("\n=== YARA Rule Selection ===")
    print("Select a YARA rule to apply:")
    for i, rule_name in enumerate(YARA_RULES.keys(), 1):
        print(f"{i}. {rule_name}")

    # Let the user select a rule
    try:
        rule_choice = int(input("Enter the number of the YARA rule to apply: ").strip())
        if rule_choice < 1 or rule_choice > len(YARA_RULES):
            print("Invalid choice. Exiting.")
            return
        rule_name = list(YARA_RULES.keys())[rule_choice - 1]
        yara_rule_content = YARA_RULES[rule_name]
    except ValueError:
        print("Invalid input. Exiting.")
        return

    print(f"\nSelected Rule: {rule_name}")
    print("Scanning file with the selected YARA rule...")

    # Validate file against the selected YARA rule
    matches = yara_scan(file_path, yara_rule_content)
    if matches is None:
        print("No matches or YARA error occurred.")
    elif len(matches) == 0:
        print("No YARA rule matches found.")
    else:
        print(f"YARA Rule Matches: {matches}")

if __name__ == "__main__":
    main()
